In [3]:
import pandas as pd
from pandas.core.computation.check import NUMEXPR_INSTALLED
import numpy as np 
from bs4 import BeautifulSoup
import requests
from datetime import datetime
import re

In [4]:
final_data = []
year = 2021
weeks = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18]

for week in weeks:
    url = f'https://www.espn.com/nfl/scoreboard/_/week/{week}/year/{year}/seasontype/2'
    soup = BeautifulSoup(requests.get(url).content, 'html.parser')

    week_data = []
    for board in soup.select('.ScoreboardScoreCell'):
        title = board.find_previous(class_='Card__Header__Title').text
        teams = [t.text for t in board.select('.ScoreCell__TeamName')]
        scores = [s.text for s in board.select('.ScoreCell__Score')] or ['-', '-']

        week_data.append((week, title, teams[0], scores[0], teams[1], scores[1]))

    final_data.extend(week_data)

df_scores_2021 = pd.DataFrame(final_data, columns=['Week', 'Date', 'Team 1', 'Score 1', 'Team 2', 'Score 2'])
df_scores_2021 = df_scores_2021.rename(columns={'Team 1': 'Away Team', 'Score 1': 'Away Score', 'Team 2': 'Home Team', 'Score 2':'Home Score'})
df_scores_2021['Final Score'] = df_scores_2021['Away Score'] + ' - ' + df_scores_2021['Home Score']
#print(df.to_markdown(index=False))
# Replace missing scores with NaN and dtype convert object to int 
df_scores_2021['Away Score'] = df_scores_2021['Away Score'].replace('-', np.nan)
df_scores_2021['Home Score'] = df_scores_2021['Home Score'].replace('-', np.nan)
df_scores_2021['Away Score'] = pd.to_numeric(df_scores_2021['Away Score'])
df_scores_2021['Home Score'] =  pd.to_numeric(df_scores_2021['Home Score'])

# Calculate total points
df_scores_2021['Total Points'] = df_scores_2021['Away Score'] + df_scores_2021['Home Score']

# Calculate winner
df_scores_2021['Winner'] = df_scores_2021.apply(lambda row: row['Away Team'] if row['Away Score'] > row['Home Score'] else row['Home Team'], axis=1)

# Calculate score differential
df_scores_2021['Away Score Differential'] = df_scores_2021['Home Score'] - df_scores_2021['Away Score']
df_scores_2021['Home Score Differential'] = df_scores_2021['Away Score'] - df_scores_2021['Home Score']
df_scores_2021['Matchup'] = df_scores_2021['Away Team'] + ' @ ' + df_scores_2021['Home Team']
df_scores_2021['Date'] = pd.to_datetime(df_scores_2021['Date'], format='%A, %B %d, %Y').dt.strftime('%Y-%m-%d')
# Use a lambda function to create the 'Home_Away_Winner' column
df_scores_2021['Home_Away_Winner'] = df_scores_2021.apply(lambda row: 'Home' 
                                                          if row['Winner'] == row['Home Team'] 
                                                          else 'Away' if row['Winner'] == row['Away Team'] else 'No Winner', axis=1)
df_scores_2021.tail()

,Week,Date,Away Team,Away Score,Home Team,Home Score,Final Score,Total Points,Winner,Away Score Differential,Home Score Differential,Matchup,Home_Away_Winner
267,18,2022-01-09,49ers,27,Rams,24,27 - 24,51,49ers,-3,3,49ers @ Rams,Away
268,18,2022-01-09,Patriots,24,Dolphins,33,24 - 33,57,Dolphins,9,-9,Patriots @ Dolphins,Home
269,18,2022-01-09,Seahawks,38,Cardinals,30,38 - 30,68,Seahawks,-8,8,Seahawks @ Cardinals,Away
270,18,2022-01-09,Panthers,17,Buccaneers,41,17 - 41,58,Buccaneers,24,-24,Panthers @ Buccaneers,Home
271,18,2022-01-09,Chargers,32,Raiders,35,32 - 35,67,Raiders,3,-3,Chargers @ Raiders,Home


In [5]:
#import consensus data for spreads 
game_dates = list(df_scores_2021['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

spreads = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        spreads.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_spreads_2021 = pd.concat(spreads, ignore_index=True)

In [6]:
#clean matchup column 
df_spreads_2021['Matchup'] = df_spreads_2021['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_spreads_2021[['Away Team', 'Home Team']] = df_spreads_2021['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_spreads_2021['Away Team'] = df_spreads_2021['Away Team'].str.strip()
df_spreads_2021['Home Team'] = df_spreads_2021['Home Team'].str.strip()
#split to create away and home spread cols 
df_spreads_2021[['Away Spread', 'Home Spread']] = df_spreads_2021['Sides'].str.split(expand=True)
#split to create away and home consensus cols 
df_spreads_2021[['Away Consensus', 'Home Consensus']] = df_spreads_2021['Consensus'].str.split(expand=True)
# convert to dtype = float for consensus cols 
df_spreads_2021['Away Consensus'] =  df_spreads_2021['Away Consensus'].str.rstrip('%').astype(int)/100
df_spreads_2021['Home Consensus'] = df_spreads_2021['Home Consensus'].str.rstrip('%').astype(int)/100
#create consensus favorite vs. underdog cols 
df_spreads_2021['Con. Spread Favorite'] = df_spreads_2021.apply(lambda x: x['Away Team'] if x['Away Consensus'] > x['Home Consensus'] else x['Home Team'], axis=1)
df_spreads_2021['Con. Spread Underdog'] = df_spreads_2021.apply(lambda x: x['Away Team'] if x['Away Consensus'] < x['Home Consensus'] else x['Home Team'], axis=1)

# Define the dictionary mapping for replacements
replacement_dict = {'LV': 'Raiders',
                    'KC': 'Chiefs',
                    'NE': 'Patriots',
                    'GB': 'Packers',
                    'NO': 'Saints',
                    'BAL': 'Ravens',
                    'SF': '49ers',
                    'NYG': 'Giants',
                    'JAC': 'Jaguars',
                    'PIT': 'Steelers',
                    'TB': 'Buccaneers',
                    'CLE': 'Browns',
                    'IND': 'Colts',
                    'PHI': 'Eagles',
                    'ARI': 'Cardinals',
                    'WAS': 'Commanders',
                    'ATL': 'Falcons',
                    'NYJ': 'Jets',
                    'SEA': 'Seahawks',
                    'MIA': 'Dolphins',
                    'HOU': 'Texans',
                    'CAR': 'Panthers',
                    'CHI': 'Bears',
                    'CIN': 'Bengals',
                    'LAR': 'Rams',
                    'BUF': 'Bills',
                    'DET': 'Lions',
                    'DEN': 'Broncos',
                    'TEN': 'Titans',
                    'MIN': 'Vikings',
                    'LAC': 'Chargers',
                    'DAL': 'Cowboys'}
# Replace the values in the DataFrame column using map()
df_spreads_2021['Away Team'] = df_spreads_2021['Away Team'].map(replacement_dict)
df_spreads_2021['Home Team'] = df_spreads_2021['Home Team'].map(replacement_dict)
df_spreads_2021['Con. Spread Favorite'] = df_spreads_2021['Con. Spread Favorite'].map(replacement_dict)
df_spreads_2021['Con. Spread Underdog'] = df_spreads_2021['Con. Spread Underdog'].map(replacement_dict)
df_spreads_2021['Matchup'] = df_spreads_2021['Away Team'] + ' @ ' + df_spreads_2021['Home Team']
df_spreads_2021.tail()

,Matchup,Date,Consensus,Sides,Picks,Indepth,Away Team,Home Team,Away Spread,Home Spread,Away Consensus,Home Consensus,Con. Spread Favorite,Con. Spread Underdog
267,Colts @ Jaguars,2022-01-09,45% 55%,-14.5 +14.5,627 755,Details,Colts,Jaguars,-14.5,+14.5,0.45,0.55,Jaguars,Colts
268,49ers @ Rams,2022-01-09,53% 47%,+3.5 -3.5,805 717,Details,49ers,Rams,+3.5,-3.5,0.53,0.47,49ers,Rams
269,Chargers @ Raiders,2022-01-09,51% 49%,-3 +3,786 747,Details,Chargers,Raiders,-3,+3,0.51,0.49,Chargers,Raiders
270,Titans @ Texans,2022-01-09,50% 50%,-11 +11,707 715,Details,Titans,Texans,-11,+11,0.50,0.50,Texans,Texans
271,Patriots @ Dolphins,2022-01-09,50% 50%,-6.5 +6.5,744 741,Details,Patriots,Dolphins,-6.5,+6.5,0.50,0.50,Dolphins,Dolphins


In [7]:
#import consensus data for spreads 
game_dates = list(df_scores_2021['Date'].unique())
uri_list = []
uri = "https://contests.covers.com/consensus/topoverunderconsensus/nfl/overall/"

for date in game_dates:
    uri_list.append(uri + date)

totals = []
for item in uri_list:
    try:
        res = requests.get(item)
        soup = BeautifulSoup(res.content, 'lxml')
        table = soup.find_all('table')[0]
        temp_df = pd.read_html(str(table))[0]  # Obtain the first table from the list
        temp_df['Date'] = re.search(r'\d{4}-\d{2}-\d{2}', item).group()  # Add the corresponding date column

        totals.append(temp_df)
    except IndexError:
        print(f"IndexError occurred for URL: {item}")
        continue
    except Exception as e:
        print(f"An error occurred for URL: {item}")
        print(f"Error details: {str(e)}")
        continue

df_totals_2021 = pd.concat(totals, ignore_index=True)
df_totals_2021.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth
267,NFL Sea Ari,2022-01-09,54 % Over 46 % Under,48.0,544 464,Details
268,NFL Pit Bal,2022-01-09,52 % Under 48 % Over,41.0,499 457,Details
269,NFL No Atl,2022-01-09,52 % Over 48 % Under,40.0,433 398,Details
270,NFL Cin Cle,2022-01-09,52 % Over 48 % Under,37.5,464 434,Details
271,NFL Ind Jac,2022-01-09,52 % Over 48 % Under,43.5,512 479,Details


In [8]:
#clean matchup column 
df_totals_2021['Matchup'] = df_totals_2021['Matchup'].str[4:].str.upper().str.replace("  ",' @ ')
# clean add home and away column 
df_totals_2021[['Away Team', 'Home Team']] = df_totals_2021['Matchup'].str.split(' @ ', expand=True)
# Remove leading/trailing whitespaces
df_totals_2021['Away Team'] = df_totals_2021['Away Team'].str.strip()
df_totals_2021['Home Team'] = df_totals_2021['Home Team'].str.strip()
# create consensus columns 
df_totals_2021['Over Consensus'] = df_totals_2021['Consensus'].str.extract(r'(\d+) % Over').astype(int)/100
df_totals_2021['Under Consensus'] = df_totals_2021['Consensus'].str.extract(r'(\d+) % Under').astype(int)/100
#create consensus favorites vs underdogs 
df_totals_2021['Con. Total Favorite'] = df_totals_2021[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] > x['Under Consensus'] else 'Under', axis=1)
df_totals_2021['Con. Total Underdog'] = df_totals_2021[['Over Consensus', 'Under Consensus']].apply(lambda x: 'Over' 
                                                                                                    if x['Over Consensus'] < x['Under Consensus'] else 'Under', axis=1)
# Replace the values in the DataFrame column using map() from our previously defined replacement_dict
df_totals_2021['Away Team'] = df_totals_2021['Away Team'].map(replacement_dict)
df_totals_2021['Home Team'] = df_totals_2021['Home Team'].map(replacement_dict)
df_totals_2021['Matchup'] = df_totals_2021['Away Team'] + ' @ ' + df_totals_2021['Home Team']
df_totals_2021.tail()

,Matchup,Date,Consensus,Total,Picks,Indepth,Away Team,Home Team,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog
267,Seahawks @ Cardinals,2022-01-09,54 % Over 46 % Under,48.0,544 464,Details,Seahawks,Cardinals,0.54,0.46,Over,Under
268,Steelers @ Ravens,2022-01-09,52 % Under 48 % Over,41.0,499 457,Details,Steelers,Ravens,0.48,0.52,Under,Over
269,Saints @ Falcons,2022-01-09,52 % Over 48 % Under,40.0,433 398,Details,Saints,Falcons,0.52,0.48,Over,Under
270,Bengals @ Browns,2022-01-09,52 % Over 48 % Under,37.5,464 434,Details,Bengals,Browns,0.52,0.48,Over,Under
271,Colts @ Jaguars,2022-01-09,52 % Over 48 % Under,43.5,512 479,Details,Colts,Jaguars,0.52,0.48,Over,Under


In [9]:
print(df_scores_2021.columns)
print(df_spreads_2021.columns)
print(df_totals_2021.columns)

Index(['Week', 'Date', 'Away Team', 'Away Score', 'Home Team', 'Home Score',
       'Final Score', 'Total Points', 'Winner', 'Away Score Differential',
       'Home Score Differential', 'Matchup', 'Home_Away_Winner'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Sides', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog'],
      dtype='object')
Index(['Matchup', 'Date', 'Consensus', 'Total', 'Picks', 'Indepth',
       'Away Team', 'Home Team', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog'],
      dtype='object')


In [10]:
# pd.merge(df1, df2, on=['Key_Column_1', 'Key_Column_2'], how='inner')
combined_df = pd.merge(df_scores_2021[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 
                                       'Winner', 'Home_Away_Winner', 'Away Team', 'Away Score', 
                                       'Away Score Differential', 'Home Team', 'Home Score', 'Home Score Differential']], 
                       df_spreads_2021[['Matchup', 'Date', 'Away Team','Away Spread', 'Home Team', 
                                        'Home Spread', 'Away Consensus', 'Home Consensus', 
                                        'Con. Spread Favorite','Con. Spread Underdog']],
                       on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                       how='inner')

In [11]:
final_df_nfl_2021 = pd.merge(combined_df[['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
                                           'Home_Away_Winner', 'Away Team', 'Away Score',
                                           'Away Score Differential', 'Home Team', 'Home Score',
                                           'Home Score Differential', 'Away Spread', 'Home Spread',
                                           'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
                                           'Con. Spread Underdog']],
                            df_totals_2021[['Matchup', 'Date', 'Away Team', 'Home Team', 'Total',
                                            'Over Consensus', 'Under Consensus',
                                            'Con. Total Favorite', 'Con. Total Underdog']],
                             on=['Matchup', 'Date', 'Away Team', 'Home Team'], 
                             how='inner')

final_df_nfl_2021['Win Marginal'] = np.abs(final_df_nfl_2021['Away Score'] - final_df_nfl_2021['Home Score'])

In [12]:
# 1) did away vs home team cover? -- Done 
# Convert 'Away Spread' and 'Home Spread' columns to numeric values
final_df_nfl_2021['Away Spread'] = pd.to_numeric(final_df_nfl_2021['Away Spread'])
final_df_nfl_2021['Home Spread'] = pd.to_numeric(final_df_nfl_2021['Home Spread'])

final_df_nfl_2021['Spread Covering Team'] = final_df_nfl_2021.apply(lambda row: row['Home Team'] 
                                                              if row['Home Score'] + row['Home Spread'] >= row['Away Score'] 
                                                              else row['Away Team'], axis=1)
final_df_nfl_2021[['Matchup', 'Final Score','Home Team', 'Home Spread', 'Home Score', 'Home Spread','Away Team', 'Away Spread', 'Spread Covering Team']].head()

,Matchup,Final Score,Home Team,Home Spread,Home Score,Home Spread,Away Team,Away Spread,Spread Covering Team
0,Cowboys @ Buccaneers,29 - 31,Buccaneers,-9.0,31,-9.0,Cowboys,9.0,Cowboys
1,Eagles @ Falcons,32 - 6,Falcons,-3.0,6,-3.0,Eagles,3.0,Eagles
2,Steelers @ Bills,23 - 16,Bills,-6.5,16,-6.5,Steelers,6.5,Steelers
3,Vikings @ Bengals,24 - 27,Bengals,3.0,27,3.0,Vikings,-3.0,Bengals
4,49ers @ Lions,41 - 33,Lions,9.5,33,9.5,49ers,-9.5,Lions


In [13]:
# 2) Did over/under hit? -- Done

# Create "Totals Covered" column
final_df_nfl_2021['Totals Covering Line'] = final_df_nfl_2021.apply(lambda row: 'Over' 
                                          if row['Total Points'] > row['Total'] 
                                          else ('Under' if row['Total Points'] < row['Total'] else 'Push'), axis=1)

# Print the updated DataFrame
final_df_nfl_2021[['Matchup', 'Total Points', 'Total', 'Totals Covering Line']].head()

,Matchup,Total Points,Total,Totals Covering Line
0,Cowboys @ Buccaneers,60,51.5,Over
1,Eagles @ Falcons,38,48.5,Under
2,Steelers @ Bills,39,48.0,Under
3,Vikings @ Bengals,51,47.0,Over
4,49ers @ Lions,74,46.0,Over


In [14]:
# 3) Did consensus fav vs underdog win? [Spread] -- Done

# Create "Consensus Spread Winner" column
final_df_nfl_2021['Consensus Spread Winner'] = final_df_nfl_2021.apply(lambda row: 'Majority' 
                                            if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] > 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] > 0.5))
                                            else 'Minority' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] < 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] < 0.5))
                                            else 'Push' if row['Spread Covering Team'] in [row['Away Team'], row['Home Team']] and
                                                          ((row['Spread Covering Team'] == row['Away Team'] and row['Away Consensus'] == 0.5) or
                                                           (row['Spread Covering Team'] == row['Home Team'] and row['Home Consensus'] == 0.5))
                                            else '', axis=1)

# Print the updated DataFrame
final_df_nfl_2021[['Away Team', 'Home Team', 'Away Consensus', 'Home Consensus', 'Consensus Spread Winner']].head()

,Away Team,Home Team,Away Consensus,Home Consensus,Consensus Spread Winner
0,Cowboys,Buccaneers,0.46,0.54,Minority
1,Eagles,Falcons,0.45,0.55,Minority
2,Steelers,Bills,0.47,0.53,Minority
3,Vikings,Bengals,0.55,0.45,Minority
4,49ers,Lions,0.64,0.36,Minority


In [15]:
# 4) Did consensus fav vs underdog win? [Totals] -- Done
# Create "Total Consensus Winner" column
final_df_nfl_2021['Total Consensus Winner'] = final_df_nfl_2021.apply(lambda row: 'Majority' 
                                                  if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] > 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] > 0.5))
                                                  else 'Minority' if ((row['Totals Covering Line'] == 'Under' and row['Under Consensus'] < 0.5) or
                                                                   (row['Totals Covering Line'] == 'Over' and row['Over Consensus'] < 0.5))
                                                  else 'Push' if (row['Under Consensus'] == 0.5) or (row['Over Consensus'] == 0.5)
                                                  else '', axis=1)

# Print the updated DataFrame
#final_df_nfl_2021[['Matchup', 'Total Points', 'Total', 'Totals Covering Line', 'Over Consensus', 'Away Consensus', 'Total Consensus Winner']]

In [16]:
final_df_nfl_2021.columns

Index(['Week', 'Date', 'Matchup', 'Final Score', 'Total Points', 'Winner',
       'Home_Away_Winner', 'Away Team', 'Away Score',
       'Away Score Differential', 'Home Team', 'Home Score',
       'Home Score Differential', 'Away Spread', 'Home Spread',
       'Away Consensus', 'Home Consensus', 'Con. Spread Favorite',
       'Con. Spread Underdog', 'Total', 'Over Consensus', 'Under Consensus',
       'Con. Total Favorite', 'Con. Total Underdog', 'Win Marginal',
       'Spread Covering Team', 'Totals Covering Line',
       'Consensus Spread Winner', 'Total Consensus Winner'],
      dtype='object')

In [17]:
final_df_nfl_2021

,Week,Date,Matchup,Final Score,Total Points,Winner,Home_Away_Winner,Away Team,Away Score,Away Score Differential,...,Total,Over Consensus,Under Consensus,Con. Total Favorite,Con. Total Underdog,Win Marginal,Spread Covering Team,Totals Covering Line,Consensus Spread Winner,Total Consensus Winner
0,1,2021-09-09,Cowboys @ Buccaneers,29 - 31,60,Buccaneers,Home,Cowboys,29,2,...,51.5,0.45,0.55,Under,Over,2,Cowboys,Over,Minority,Minority
1,1,2021-09-12,Eagles @ Falcons,32 - 6,38,Eagles,Away,Eagles,32,-26,...,48.5,0.54,0.46,Over,Under,26,Eagles,Under,Minority,Minority
2,1,2021-09-12,Steelers @ Bills,23 - 16,39,Steelers,Away,Steelers,23,-7,...,48.0,0.56,0.44,Over,Under,7,Steelers,Under,Minority,Minority
3,1,2021-09-12,Vikings @ Bengals,24 - 27,51,Bengals,Home,Vikings,24,3,...,47.0,0.49,0.51,Under,Over,3,Bengals,Over,Minority,Minority
4,1,2021-09-12,49ers @ Lions,41 - 33,74,49ers,Away,49ers,41,-8,...,46.0,0.49,0.51,Under,Over,8,Lions,Over,Minority,Minority
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
250,18,2022-01-09,49ers @ Rams,27 - 24,51,49ers,Away,49ers,27,-3,...,45.5,0.63,0.37,Over,Under,3,49ers,Over,Majority,Majority
251,18,2022-01-09,Patriots @ Dolphins,24 - 33,57,Dolphins,Home,Patriots,24,9,...,40.5,0.57,0.43,Over,Under,9,Dolphins,Over,Push,Majority
252,18,2022-01-09,Seahawks @ Cardinals,38 - 30,68,Seahawks,Away,Seahawks,38,-8,...,48.0,0.54,0.46,Over,Under,8,Seahawks,Over,Minority,Majority
253,18,2022-01-09,Panthers @ Buccaneers,17 - 41,58,Buccaneers,Home,Panthers,17,24,...,42.0,0.59,0.41,Over,Under,24,Buccaneers,Over,Majority,Majority


In [18]:
final_df_nfl_2021.to_csv('final_df_nfl_2021.csv')